# 마르코프 체인
![](http://aidev.co.kr/files/attach/images/1318/116/001/92016e2a4b20026ef88a4b57df90d648.png)


* 이전 상태가 다음 상태의 영향을 주는것으로 1개 또는 N개의 이전 상태가 주어졌을 때 다음상태가 나올 확률을 계산하여 실절적인 의미의 연관성을 생각하지 않고 문장을 조합.
* 조금 이상한 문장이 만들어지는 경우가 있음. 

## 문장을 만드는 과정
1. 문장을 단어로 분할
2. 단어의 전후 연결을 딕셔너리에 등록
3. 사전을 사용해 임의의 문장을 생성 

## 예제 
* 각 문장의 형태소 분석 
 * 개|도|닷새|가|되면|주인|을|안다.
 * 기르던|개|에게|다리|가|물렸다.
 * 닭|쫒던|개|지붕|쳐다|보듯|한다.
 * 똥|묻은|개|가|겨|묻은|개|나무란다
 
* 단어의 전후 딕셔너리 등록 
 * 개 : 가,에게,지붕,도
 * 가 : 되면,겨
 * 되면 : 주인
 * 주인 : 주인을 안다. 
 
* 문장생성
 * "개가 되면 주인을 안다."



In [1]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import urllib.request
import os, json, random


# 마르코프 체인 딕셔너리 만들기 --- (※1)
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == ".":
            tmp = ["@"]
            continue
    return dic


# 딕셔너리에 데이터 등록하기 --- (※2)
def set_word3(dic, s3):
    w1, w2, w3 = s3
    
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
        
    dic[w1][w2][w3] += 1


In [ ]:
# 문장 만들기 --- (※3)
def make_sentence(dic):
    ret = []
    if not "@" in dic: return "no dic"
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == ".": break
        w1, w2 = w2, w3
    ret = "".join(ret)
    # 결과물 
    #'희번덕이는섬진강저켠은전라도땅이켠은경상도땅너그럽게그어진능선은확실한윤곽을드러낸다.'
    # 띄어쓰기
    params = urllib.parse.urlencode({"_callback": "", "q": ret})
    # 네이버 맞춤법 검사기를 사용합니다.
    data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/dcontent/spellchecker.nhn?" + params)
    data = data.read().decode("utf-8")[1:-2]
    data = json.loads(data)
    data = data["message"]["result"]["html"]
    data = BeautifulSoup(data, "html.parser").getText()
    # 리턴
    return data


def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))


In [2]:
# 문장 읽어 들이기 --- (※4)
dict_file = "markov-toji.json"
if not os.path.exists(dict_file):
    # 토지 텍스트 파일 읽어 들이기
    fp = codecs.open("BEXX0003.txt", "r", encoding="utf-16")
    soup = BeautifulSoup(fp, "html.parser")
    body = soup.select_one("body > text")
    text = body.getText()
    text = text.replace("…", "")  # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결
    # 형태소 분석
    twitter = Twitter()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단 마침표는 포함)
        if not word[1] in ["Punctuation"]:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file, "w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file, "r"))

# 문장 만들기 --- (※6)
for i in range(10):
    s = make_sentence(dic)
    print(s)
    print("---")

비싼 약도 아니고 귀가 잘린 데다 삭발을 겁내어 이젠 양반들도 일어서서 먼빛의 말뿐이었는데 그래서 그를괴롭혀 왔었다 안 돼 응원의 전령을 받았을 인지 김훈장에게 위로가 된 입을 막고 흙의 양분을독차지하더니 이제 목숨을 다한 보리 뿌리는썩어서 뭉개진 문등이 탈을 쓴 허위 대가 헌칠한 용이는부지런해졌으며 강청댁은 낫을 들었는지요 치어 매는 말하였소.
---
행랑 구석진 방에서 욕을 했기 웅담인가 강포수 코 고는 소리를 한번 질렀다.
---
허한 구석이 있어도 소용없다.
---
거짓말한다 캐서 너 봉순아 어서 아기씨를 속 있다 그 말심이구마 헛나 재물과 목숨 지키려고 상것들에게 존경심을갖게 되었다.
---
박복할 수 없제 서울 그 나리는 왜 안 와 하나 잇몸이간지러버서 어쩔 수 없네 방자 청하는 거나 주게 그 말이지.
---
샛서방이 있어갔다가 밤늦게 오니 코를잘렸느니 그게 아니고 그래 부딪쳐본 거라 생각했다.
---
불꽃은 한번 자지러졌다가 밝아왔다 갔다 하던 용이는 새가있었다면 그것은 강청댁 눈길에 쫓겨 도망 온 것이다 누를 끼칠까보아 조심스럽게 불평을 했던지 윤 씨는만족한 듯 뇌더니 방에서 면경을 풂 속에 넣는다.
---
건달풍의 사내가 못할 개인데 숭년하고 무신 상관이 없다가서희를 상대로 놀이하던 것을 본 막딸네의 말대로 그것은김서방 뒤에 숨어있는 가배요 행색이 어떻든고 별당아씨를닮았다.
---
상사벵걸린쪽이숨넘어가기전에없었던일이다월선 어미 얼굴이 하얗게 핀 석류꽃을 올려다본다.
---
옆방에서는 남편의 요지부동한 고집 때문이었다더구먼요 그보다 낼 아침에 그들이 모여지는 이유를 그들의 놀이는 이제부터 달뜨기를 기다려 강가 모래밭에서 호작 거리는 물을끌어들여 전등이란 걸 켰다니 희한한 꿈을꾸었나 목소리에도 날을 세우며 삼 월이는사시나무 떨듯 떨고 있는 월선을 노려보면서 제 귀에 회초리를버린 윤 씨는 별당 근처를 벌 한 마리 미움 속에 가두고 난 뒤 안심한 목동이 주막에서 술을 부어 서로 권한다.
---


1. 형태소 분석 
***
"제1편 어둠의 발소리"
***
2. 딕셔너리 등록 
***
{'@': {'제': {'1': 1}}, '제': {'1': {'편': 1}}, '1': {'편': {'어둠': 1}}, '편': {'어둠': {'의': 1}}, '어둠': {'의': {'발': 1}}, '의': {'발': {'소리': 1}}, '발': {'소리': {'서': 1}}, '소리': {'서': {'序': 1}}}...
***
3. 문장 생성
***
렌덤으로 첫문장가져와서 해당 맞는 문장가져오기 
***